<a href="https://colab.research.google.com/github/subandwho/tensorflow/blob/main/NLP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-330af774-c443-341c-0ff8-16054fb423f8)


In [9]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-19 11:04:39--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-19 11:04:39 (78.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [10]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [11]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-07-19 11:04:39--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 2607:f8b0:4023:c06::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-07-19 11:04:39 (146 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [12]:
unzip_data('nlp_getting_started.zip')

In [13]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
train_df_shuffle = train_df.sample(frac = 1, random_state = 42)
train_df_shuffle.head(5)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [15]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [17]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffle[['text', 'target']][random_index:random_index+5].itertuples():
   _, text, target = row
   print(f"Target: {target}", "{real_disaster}" if target >0 else "{not disaster}")
   print(f"Text:\n{text}\n")
   print("===\n")

Target: 0 {not disaster}
Text:
I ran with earbuds in which I now realize means I probably didn't politely say hi to Jared but more or less screamed at him ??????

===

Target: 0 {not disaster}
Text:
Bed time. Don't wake me up unless revolution or Armageddon start.

===

Target: 0 {not disaster}
Text:
@marksmaponyane Hey!Sundowns were annihilated in their previous meeting with Celtic.Indeed its an improvement.

===

Target: 0 {not disaster}
Text:
@BarackObama Senator John McCainÛªs Whoops Moment : Photographed Chilling With ISISÛ_ http://t.co/JNrOMsE1Z2 @acmilan http://t.co/w6Yu7Qs4CV

===

Target: 0 {not disaster}
Text:
Next Man Up---AH SCREW THIS! I'm so tired of injuries.  

What happened to Camp Cupcake? More like Camp Cramp and Break.

===



In [18]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffle['text'].to_numpy(),
                                                                            train_df_shuffle['target'].to_numpy(),
                                                                                             test_size = 0.1, random_state=42)

In [19]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [20]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [21]:
text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize ="lower_and_strip_punctuation",
                                    split ="whitespace",
                                    ngrams=None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens = True)

In [22]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [23]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [24]:
text_vectorizer.adapt(train_sentences)

In [25]:
random_sentence = random.choice(train_sentences)
text_vectorizer([random_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1773,  439,    3, 4067,    1,  123,    4,    2, 1540,  594,  104,
           5,   10,   51,   76]])>

In [26]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [27]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ('clf', MultinomialNB())
])
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [30]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baselinemodel score: {baseline_score*100:.2f}")

Baselinemodel score: 79.27


In [33]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [37]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall":model_recall,
                   "f1":model_f1}
  return model_results                 

In [39]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}